<a href="https://colab.research.google.com/github/diegotluz/projects-colab/blob/main/Exemplo_CNN_Dataset_Digitos_Reais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Classificação de Imagens Reais

Baixe o seguinte arquivo: https://www.dropbox.com/s/sofyakylqodcrd3/Digitos.zip?dl=0

Faça o upload desse aquivo para a pasta principal de arquivos do Google Colab

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,AveragePooling2D
from keras.utils import img_to_array , load_img
import numpy as np

from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/Digitos.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Datasets')

In [ ]:
train_datagen = ImageDataGenerator(validation_split=0.20)
train_generator = train_datagen.flow_from_directory(
    '/content/Datasets/Digitos/Treinamento',
    batch_size=200,
    color_mode = 'grayscale',
    subset='training',
    target_size=(30, 30))


In [ ]:
valid_generator = train_datagen.flow_from_directory(
    '/content/Datasets/Digitos/Treinamento',
    batch_size=200,
    color_mode = 'grayscale',
    subset='validation',
    target_size=(30, 30))

In [ ]:
#Visualizando
x,y = next(train_generator)
for i in range(0,1):
    image = x[i]
    plt.imshow(image[:,:,0], cmap=plt.get_cmap('gray'))
    plt.show()

In [ ]:
x.shape

In [ ]:
image.shape

In [ ]:
y[0]

In [ ]:
num_classes = 10

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


In [ ]:
# Cria o modelo
model = Sequential()

#Convolução 2D com função de ativação Rectified Linear Units 64 kernels/Pesos (filtros)
model.add(Conv2D(64, (5, 5), input_shape=(30,30,1), activation='relu'))
print( model.output_shape)
#Camada de Pooling
model.add(AveragePooling2D(pool_size=(2, 2)))
print( model.output_shape)

#Convolução 2D com função de ativação Rectified Linear Units 128 kernels/Pesos (filtros)
model.add(Conv2D(128, (3, 3), activation='relu'))
print( model.output_shape)
#Camada de Pooling
model.add(AveragePooling2D(pool_size=(2, 2)))
print( model.output_shape)

#Remove 30% dos dados de entrada aleatoriamente
model.add(Dropout(0.3))
#Converte o conjunto de imagens e um vetor unidimensional para a entrada da rede neural totalmente conectada
model.add(Flatten())
print( model.output_shape)

model.add(Dense(64, activation='relu'))
print( model.output_shape)
model.add(Dense(64, activation='relu'))
print( model.output_shape)
model.add(Dense(num_classes, activation='softmax'))
print( model.output_shape)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
mcp_save = ModelCheckpoint('model_best.keras', monitor='val_accuracy', mode= 'auto', save_best_only=True, verbose =1)
#reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

model.summary()

model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10, callbacks=[mcp_save])

In [ ]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    '/content/Datasets/Digitos/Teste',
    batch_size=1,
    color_mode = 'grayscale',
    target_size=(30, 30))

In [ ]:
scores = model.evaluate(test_generator)
print(scores)

In [ ]:
predictions = model.predict(test_generator)
print(predictions)

In [ ]:
res1 = np.argmax(predictions[0,:])
print(res1)

Como Fazer a predição de uma imagem PNG?

In [ ]:
#fileimg = r'D:\digito_7_2SAN.png'
fileimg = r'/content/Datasets/Digitos/Teste/7/digito_2_36-49-513863.png'

img = load_img(fileimg , color_mode = "grayscale", target_size=(30, 30))

#
x = img_to_array(img)

plt.imshow(x[:,:,0], cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
x.shape

In [ ]:
x = np.expand_dims(x, axis=0)
x.shape

In [ ]:
y_pred = model.predict(x)

res = np.argmax(y_pred)
print(res)

In [ ]:
y_pred

In [ ]:
train_generator.class_indices